In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average #this used to import plot subj sig and nonsig, and plot electrodes from mri directly, but in order to save aaron's changes, i put them into jim_mri
from src.analysis.vis.jim_mri import plot_subj_sig_and_nonsig, _plot_electrodes, get_sub_dir
from ieeg.io import raw_from_layout, get_data
# from utils import load_sig_chans, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict, channel_names_to_indices
from src.analysis.utils.general_utils import *

import json
import pyvista as pv
pv.global_theme.allow_empty_mesh = True

In [ ]:
# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']

# leave out D107A for now cuz it's breaking my code. TODO: fix plotting code to work with D107A.
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0110', 'D0116', 'D0117', 'D0121']

get significant channels per roi

In [ ]:
epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"

config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)


# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071


KeyboardInterrupt: 

### plot single subject's significant electrodes

In [3]:
test = subject_to_info('D65')

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


In [4]:
subj = "D0100"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10.json'
stim_sig_chans = load_sig_chans(stim_filename)
# resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response_fixationCrossBase_1sec_mirror.json'
# resp_sig_chans = load_sig_chans(stim_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
# plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_sigStim.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

Loaded significant channels for subject D0100
Using pyvistaqt 3d backend.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


In [ ]:
subj = "D0063"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'
congruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_congruent.json'
incongruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_incongruent.json'
repeat_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_repeat.json'
switch_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_switch.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)
congruent_sig_chans = load_sig_chans(congruent_filename)
incongruent_sig_chans = load_sig_chans(incongruent_filename)
repeat_sig_chans = load_sig_chans(repeat_filename)
switch_sig_chans = load_sig_chans(switch_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=congruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Congruent', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=incongruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Incongruent', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=repeat_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Repeat', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=switch_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Switch', background='black', color=(1, 1, 0))


turn the sig electrodes per subject roi loaded into a plottable format list

In [ ]:
import re

def format_electrodes_subject_ids(data_dict):
    """
    Formats electrode names and subject IDs by removing leading zeros from the numerical part
    of the subject IDs and concatenating the formatted subject ID with electrode names.

    Parameters:
    - data_dict: A dictionary with the structure {roi: {subject_id: [electrodes]}}.

    Returns:
    - A list of formatted strings in the "Dxx-Electrode" format.
    """
    formatted_list = []
    
    # Define a regex pattern to capture parts of the subject_id
    # Pattern Explanation:
    # ^D          : Starts with 'D'
    # (0*)        : Capture any leading zeros
    # (\d+)       : Capture one or more digits (the numerical part)
    # ([A-Za-z]*) : Capture any trailing letters (optional)
    pattern = re.compile(r'^D(0*)(\d+)([A-Za-z]*)$')
    
    for roi, subjects in data_dict.items():
        for subject_id, electrodes in subjects.items():
            match = pattern.match(subject_id)
            if match:
                leading_zeros, numbers, letters = match.groups()
                # Remove leading zeros by converting to int and back to string
                # This automatically removes any leading zeros
                formatted_numbers = str(int(numbers)) if numbers else '0'
                # Reconstruct the formatted subject ID
                formatted_subject_id = f"D{formatted_numbers}{letters}"
            else:
                # If the subject_id doesn't match the pattern, keep it unchanged
                # Optionally, you can choose to skip or raise an error
                print(f"Warning: Subject ID '{subject_id}' does not match the expected pattern. It will be kept unchanged.")
                formatted_subject_id = subject_id
            
            # Attach each electrode to the formatted subject ID
            for electrode in electrodes:
                formatted_electrode = f"{formatted_subject_id}-{electrode}"
                formatted_list.append(formatted_electrode)
    
    return formatted_list

# Use the function on your loaded dictionary
plottable_sig_electrodes_per_subject_roi = format_electrodes_subject_ids(sig_electrodes_per_subject_roi)

# And also for all electrodes per subject roi
plottable_all_electrodes_per_subject_roi = format_electrodes_subject_ids(electrodes_per_subject_roi)

# Print or use the resulting lists as needed
print(plottable_sig_electrodes_per_subject_roi)
print(plottable_all_electrodes_per_subject_roi)

['D57-RAI6', 'D57-RPI15', 'D57-RPI14', 'D57-RAMF13', 'D57-RAMF14', 'D59-LMMF9', 'D59-LMMF11', 'D59-LMMF10', 'D59-LMMF12', 'D59-LPSF16', 'D63-LMSF5', 'D63-RAMF12', 'D63-RAMF13', 'D63-RMMF13', 'D63-RMMF14', 'D63-RAI4', 'D63-RAI6', 'D63-RAI11', 'D63-RMMF10', 'D65-RASF14', 'D65-RASF15', 'D65-RI7', 'D71-RFO14', 'D71-RFO16', 'D71-RIA4', 'D71-RIP6', 'D71-RIA5', 'D71-RIA11', 'D71-RIA12', 'D71-RIA13', 'D71-RIA14', 'D71-RIA16', 'D71-RIP16', 'D90-RIA6', 'D90-RIA11', 'D90-RIA12', 'D94-LFAM8', 'D94-LFAM9', 'D94-LFAM10', 'D94-LFPM10', 'D94-LFPM11', 'D94-LFPM12', 'D94-LFPI10', 'D94-LPAI10', 'D94-LFAI9', 'D94-LFAI10', 'D94-LIA11', 'D102-RFO13', 'D102-RFO14', 'D102-RFAM15', 'D102-RFAI3', 'D103-LFAM8', 'D103-LFAM9', 'D103-LAI13', 'D103-LAI14', 'D103-LAI4', 'D103-LAI7', 'D103-LAI8', 'D103-LFAI4', 'D110-LASF10', 'D110-LINS4']
['D57-RAI6', 'D57-RAI12', 'D57-RAI13', 'D57-RAI14', 'D57-RAI15', 'D57-RAI16', 'D57-RPI15', 'D57-RPI14', 'D57-RAMF10', 'D57-RAMF11', 'D57-RAMF12', 'D57-RAMF13', 'D57-RAMF14', 'D57-RAI

just plot one electrode for poster 4/8

In [10]:
fig = plot_on_average(subjects, picks=['D57-RAI6'], rm_wm=False, hemi='both', color=(1,1,0), size=0.35, transparency=0.4, label_every=1)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

OSError: [WinError 1006] The volume for a file has been externally altered so that the opened file is no longer valid: 'C:\\Users\\jz421\\Box\\ECoG_Recon\\D59\\mri\\transforms\\talairach.xfm'

plot all lpfc sig electrodes

In [14]:
fig = plot_on_average(subjects, picks=plottable_sig_electrodes_per_subject_roi, rm_wm=False, hemi='both', color=(1,1,0), size=0.45, transparency=0.4)

Using pyvistaqt 3d backend.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [16]:
fig = plot_on_average(subjects, picks=plottable_all_electrodes_per_subject_roi, rm_wm=False, hemi='both', color=(0,0.5,0), size=0.45, transparency=0.4)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [ ]:
plot_on_average(picks=plottable_all_electrodes_per_subject_roi, hemi='both', color=(1,0.5,0))

### plot all significant electrodes on average brain

In [ ]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"
print(layout.derivatives)
print(layout.derivatives.keys())

base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

# subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    
    filename = f'{base_path}\\{subj}\\sig_chans_{subj}_{epochs_root_file}.json'
    sig_chans = load_sig_chans(filename)

    # Add them to the dictionary, tagged with the subject without zeroes
    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = sig_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

picks = [f'D{k[1:].zfill(4)}-{v}' for k in allSigElectrodes.keys() for v in allSigElectrodes[k]]
# sinfos = [subject_to_info(f'D{int(subj[1:])}') for subj in subjects]

plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.35)
# plot_on_average(allElectrodes, hemi='both', color=(0,0.5,0), size=0.35)

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 17 | Sessions: 0 | Runs: 65}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 17 | Sessions: 0 | Runs: 65})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Loaded significant channels for subject D0090
Loaded significant channels for subject D0094


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Loaded significant channels for subject D0100
Loaded significant channels for subject D0102


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Loaded significant channels for subject D0103
Loaded significant channels for subject D0110


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:872: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [ ]:
allSigElectrodes

### plot all significant electrodes, then plot all electrodes

In [50]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

# subjects = layout.get(return_type="id", target="subject")

subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    # if subj == "D0077":
    #     continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    # combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # delete this line once we have responses for everyone
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

# apparently this doesn't work because the inputs are dicts and plot on average doesn't like dict inputs.
# use Aaron's code below that converts from these dicts to a list of electrode indices 3/25. 
# fig = plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.45)
# plot_on_average(allElectrodes, hemi='both', color=(1,1,1), fig=fig)

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jz421\\Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs\\D0107\\sig_chans_D0107_Stimulus_fixationCrossBase_1sec_mirror.json'

aaron's way of plotting

In [8]:
allSigElectrodes
subjects = []
length = 0
elecs = []
for subject, electrodes in allSigElectrodes.items():
    info = subject_to_info(subject)
    for electrode in electrodes:
        elecs.append(length + info['ch_names'].index(electrode))
    length += len(info['ch_names'])
    subjects.append(subject)

fig = plot_on_average(subjects, picks='D57-RMMF14', rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4, transparency=0.2)
# plot_on_average(subjects, rm_wm=False, hemi='both', color=(1, 1, 1), fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


ValueError: No channels match the selection.

plot sig for congruency at any time window and also sig for interaction between congruency and congruency proportion at any time window 3/25  

this is for all electrodes

In [23]:
LAB_root = None
channels = None
full_trial_base = False
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# congruency_congruencyProportion_structure_file_path = os.path.join(save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
# with open(congruency_congruencyProportion_structure_file_path, 'r') as file:
#     congruency_congruencyProportion_structure = json.load(file)

# switchType_switchProportion_structure_file_path = os.path.join(save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
# with open(switchType_switchProportion_structure_file_path, 'r') as file:
#     switchType_switchProportion_structure = json.load(file)

congruency_switchType_structure_file_path = os.path.join(save_dir, 'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(congruency_switchType_structure_file_path, 'r') as file:
    congruency_switchType_structure = json.load(file)

this is just for lpfc 4/1

In [25]:
LAB_root = None
channels = None
full_trial_base = False
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
roi = 'lpfc'

# # congruency_congruencyProportion_structure_file_path = os.path.join(save_dir, f'congruency_congruencyProportion_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
# # with open(congruency_congruencyProportion_structure_file_path, 'r') as file:
# #     congruency_congruencyProportion_structure = json.load(file)

# # switchType_switchProportion_structure_file_path = os.path.join(save_dir, f'switchType_switchProportion_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
# # with open(switchType_switchProportion_structure_file_path, 'r') as file:
# #     switchType_switchProportion_structure = json.load(file)

# old_congruency_switchType_structure_file_path = os.path.join(save_dir, f'congruency_switchType_ANOVAwithinElectrodes_significantEffectsStructure_{roi}.txt')
# with open(old_congruency_switchType_structure_file_path, 'r') as file:
#     old_congruency_switchType_structure = json.load(file)

In [ ]:
congruency_switchType_structure_file_path = os.path.join(save_dir, 'stimulus_main_effect_conditions_significantEffectsStructure_lpfc.txt')
with open(congruency_switchType_structure_file_path, 'r') as file:
    congruency_switchType_structure = json.load(file)

whole brain plotting setup (old as of 4/1, delete once the other method works for whole brain plotting)

In [21]:
test = subject_to_info('D63')

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


In [32]:
congruency_elecs = []
switchType_elecs = []
congruency_congruencyProportion_elecs = []
switchType_switchProportion_elecs = []
congruency_switchType_elecs = []

subjects = []
added_elecs_congruency = set()  # To keep track of added electrodes for congruency
added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

added_elecs_switchType = set()  # To keep track of added electrodes for switch type
added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion

added_elecs_congruency_switchType = set()  # To keep track of added electrodes for congruency_switchType

def process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, effect_name, elecs_list, added_elecs_set):
    """A helper function to process electrode effects and update lists and sets accordingly."""
    for time_window, effects in time_windows.items():
        for effect_and_p_value in effects:
            if effect_and_p_value[0] == effect_name and (subj_without_zeroes, electrode) not in added_elecs_set:
                elecs_list.append(length + info['ch_names'].index(electrode))
                added_elecs_set.add((subj_without_zeroes, electrode))
                
# Assuming all structures have the same subjects...
length = 0
# structure_name = 'congruency_congruencyProportion_structure'
# structure_name = 'switchType_switchProportion_structure'
structure_name = 'congruency_switchType_structure'

if structure_name == 'congruency_congruencyProportion_structure':
    structure = congruency_congruencyProportion_structure
elif structure_name == 'switchType_switchProportion_structure':
    structure = switchType_switchProportion_structure
elif structure_name == 'congruency_switchType_structure':
    structure = congruency_switchType_structure

for subject in structure:
    subj_without_zeroes = "D" + subject[1:].lstrip('0')
    print(subj_without_zeroes)
    info = subject_to_info(subj_without_zeroes)
    print('got info')
    # Ensure each subject is only added once
    if subj_without_zeroes not in subjects:
        subjects.append(subj_without_zeroes)

    current_subject_length = len(info['ch_names'])

    # # Process congruency and congruency_congruencyProportion effects - OLD AS OF 10/1
    # if structure_name == 'congruency_congruencyProportion_structure':
    #     electrodes_dict_congruency_congruencyProportion = congruency_congruencyProportion_structure[subject]
    #     for electrode, time_windows in electrodes_dict_congruency_congruencyProportion.items():
    #         process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency)
    #         process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'congruency:congruencyProportion', congruency_congruencyProportion_elecs, added_elecs_congruency_congruencyProportion)

    # # Process switchType and switchType:switchProportion effects
    # if structure_name == 'switchType_switchProportion_structure':
    #     electrodes_dict_switchType_switchProportion = switchType_switchProportion_structure.get(subject, {})
    #     for electrode, time_windows in electrodes_dict_switchType_switchProportion.items():
    #         process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType)
    #         process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'switchType:switchProportion', switchType_switchProportion_elecs, added_elecs_switchType_switchProportion)

    # Process congruency and switch type interaction effect
    if structure_name == 'congruency_switchType_structure':
        electrodes_dict_congruency_switchType = congruency_switchType_structure.get(subject, {})
        for electrode, rois in electrodes_dict_congruency_switchType.items():
            for roi, time_windows in rois.items():
                for time_window, effects in time_windows.items():
                    process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency) # this is a duplicate so don't run this if already have congruency_elecs
                    process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType) # this is a duplicate so don't run this if already have switchType_elecs
                    process_effects_whole_brain_analysis(electrode, time_windows, subj_without_zeroes, info, length, 'congruency:switchType', congruency_switchType_elecs, added_elecs_congruency_switchType)

    length += current_subject_length  # Update length after processing each subject

D57
got info
D59
got info
D63
got info
D65
got info
D71
got info
D94


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

got info
D102
got info
D103
got info
D107A
got info
D110
got info


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


adapt whole brain setup so it works with roi plotting too 4/1

In [12]:
# # Assuming all structures have the same subjects...
# subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

# congruency_elecs = []
# switchType_elecs = []
# congruency_congruencyProportion_elecs = []
# switchType_switchProportion_elecs = []
# congruency_switchType_elecs = []

# added_elecs_congruency = set()  # To keep track of added electrodes for congruency
# added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

# added_elecs_switchType = set()  # To keep track of added electrodes for switch type
# added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion

# added_elecs_congruency_switchType = set()  # To keep track of added electrodes for congruency_switchType

# def process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, effect_name, elecs_list, added_elecs_set):
#     """A helper function to process electrode effects and update lists and sets accordingly."""
#     for time_window, effects in time_windows.items():
#         for effect_and_p_value in effects:
#             if effect_and_p_value[0] == effect_name and (subject_without_zeroes, electrode) not in added_elecs_set:
#                 elecs_list.append(length + info['ch_names'].index(electrode))
#                 added_elecs_set.add((subject_without_zeroes, electrode))
                
# length = 0
# ch_names = []
# for subject in subjects:
#     subject_without_zeroes = "D" + subject[1:].lstrip('0')
#     info = subject_to_info(subject_without_zeroes)

#     ch_names += ['-'.join([subject, n]) for n in info['ch_names']] #get a list of all channel names
#     current_subject_length = len(info['ch_names'])

#     # # Process congruency and congruency_congruencyProportion effects
#     # electrodes_dict_congruency_congruencyProportion = congruency_congruencyProportion_structure[subject]
#     # for electrode, time_windows in electrodes_dict_congruency_congruencyProportion.items():
#     #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency)
#     #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency_congruencyProportion', congruency_congruencyProportion_elecs, added_elecs_congruency_congruencyProportion)

#     # # Process switchType and switchType:switchProportion effects
#     # electrodes_dict_switchType_switchProportion = switchType_switchProportion_structure.get(subject, {})
#     # for electrode, time_windows in electrodes_dict_switchType_switchProportion.items():
#     #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType)
#     #     process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType:switchProportion', switchType_switchProportion_elecs, added_elecs_switchType_switchProportion)

#     # Process congruency and switch type interaction effect
#     if congruency_switchType_structure and subject in congruency_switchType_structure:
#         electrodes_dict_congruency_switchType = congruency_switchType_structure.get(subject, {})
#         for electrode, time_windows in electrodes_dict_congruency_switchType.items():
#             process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency) # delete this line if i use congruency_congruencyProportion
#             process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType) # delete this line if i use switchType_switchProportion
#             process_effects_whole_brain_analysis(electrode, time_windows, subject_without_zeroes, info, length, 'congruency_switchType', congruency_switchType_elecs, added_elecs_congruency_switchType)


#     length += current_subject_length  # Update length after processing each subject

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


plot unique congruency, unique congruency and congruency proportion interaction, and also their overlap

In [34]:
len(switchType_elecs)

28

In [35]:
len(congruency_elecs)

23

plot congruency, congruency proportion, and mixed electrodes

In [37]:
if structure_name == 'congruency_congruencyProportion_structure':

    # Convert lists to sets for set operations
    congruency_elecs_set = set(congruency_elecs)
    congruency_congruencyProportion_elecs_set = set(congruency_congruencyProportion_elecs)

    # Find overlapping electrodes (present in both lists)
    overlap_congruencyEffectAndLWPC_elecs = congruency_elecs_set.intersection(congruency_congruencyProportion_elecs_set)

    # Find unique electrodes in congruency_elecs
    unique_congruency_elecs = congruency_elecs_set - congruency_congruencyProportion_elecs_set

    # Find unique electrodes in congruency_congruencyProportion_elecs
    unique_congruency_congruencyProportion_elecs = congruency_congruencyProportion_elecs_set - congruency_elecs_set

    # Convert sets back to lists if necessary for plotting functions
    overlap_congruencyEffectAndLWPC_elecs_list = list(overlap_congruencyEffectAndLWPC_elecs)
    unique_congruency_elecs_list = list(unique_congruency_elecs)
    unique_congruency_congruencyProportion_elecs_list = list(unique_congruency_congruencyProportion_elecs)

    transparency = 0.4

    # Plotting
    # Plot unique electrodes for congruency in red
    fig = plot_on_average(subjects, picks=unique_congruency_elecs_list, rm_wm=False, hemi='both', color=(1,0,0), size=0.35, label_every=None, transparency=transparency)

    # Plot unique electrodes for congruency_congruencyProportion in hot pink
    fig = plot_on_average(subjects, picks=unique_congruency_congruencyProportion_elecs_list, rm_wm=False, hemi='both', color=(1,0.2,0.6), size=0.35, fig=fig, transparency=transparency)

    # Plot overlapping electrodes in black
    fig = plot_on_average(subjects, picks=overlap_congruencyEffectAndLWPC_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=transparency)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

IndexError: list index out of range

plot switch type and switch type:switchProportion

In [93]:
if structure_name == 'switchType_switchProportion_structure':
    # Convert lists to sets for set operations
    switchType_elecs_set = set(switchType_elecs)

    switchType_switchProportion_elecs_set = set(switchType_switchProportion_elecs)

    # Find overlapping electrodes (present in both lists)
    overlap_switchCostAndLWPS_elecs = switchType_elecs_set.intersection(switchType_switchProportion_elecs_set)

    # Find unique electrodes in switch type elecs
    unique_switchType_elecs = switchType_elecs_set - switchType_switchProportion_elecs_set

    # Find unique electrodes in congruency_congruencyProportion_elecs
    unique_switchType_switchProportion_elecs = switchType_switchProportion_elecs_set - switchType_elecs_set

    # Convert sets back to lists if necessary for plotting functions
    overlap_switchCostAndLWPS_elecs_list = list(overlap_switchCostAndLWPS_elecs)
    unique_switchType_elecs_list = list(unique_switchType_elecs)
    unique_switchType_switchProportion_elecs_list = list(unique_switchType_switchProportion_elecs)

    transparency = 0.6
    # Plotting
    # Plot unique electrodes for switch type in one color
    fig = plot_on_average(subjects, picks=unique_switchType_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 1), size=0.35, label_every=None, transparency=transparency)

    # Plot unique electrodes for switchType:switchProportion in another color
    fig = plot_on_average(subjects, picks=unique_switchType_switchProportion_elecs_list, rm_wm=False, hemi='both', color=(0.5, 0.5, 0.5), size=0.35, fig=fig, transparency=transparency)

    # Plot overlapping electrodes in black
    fig = plot_on_average(subjects, picks=overlap_switchCostAndLWPS_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=transparency)

Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D57\elec_recon\D57_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D59\elec_recon\D59_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D63\elec_recon\D63_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D71\elec_recon\D71_elec_locations_RAS_brainshifted.txt


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D94\elec_recon\D94_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D102\elec_recon\D102_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D103\elec_recon\D103_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D57\elec_recon\D57_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D59\elec_recon\D59_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D63\elec_recon\D63_elec_locations_RAS_brainshifted.txt


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D71\elec_recon\D71_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D94\elec_recon\D94_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D102\elec_recon\D102_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D103\elec_recon\D103_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D57\elec_recon\D57_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D59\elec_recon\D59_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D63\elec_recon\D63_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C:\Users\jz421\Box\ECoG_Recon\D71\elec_recon\D71_elec_locations_RAS_brainshifted.txt
Attempting to read electrode file: C

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:760: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [39]:
if structure_name == 'congruency_switchType_structure':

    congruency_elecs_set = set(congruency_elecs)
    switchType_elecs_set = set(switchType_elecs)
    congruency_switchType_elecs_set = set(congruency_switchType_elecs)

    # Find overlapping electrodes (present in both lists). Then also find union with the interaction effect.
    overlap_congruencyEffectAndSwitchCost_elecs = switchType_elecs_set.intersection(congruency_elecs_set)
    overlap_and_interaction_congruencyEffectAndSwitchCost_elecs = (overlap_congruencyEffectAndSwitchCost_elecs | congruency_switchType_elecs_set)

    # Find unique electrodes in switchType_elecs
    unique_switchType_elecs = switchType_elecs_set - congruency_elecs_set - congruency_switchType_elecs_set

    # Find unique electrodes in congruency_elecs
    unique_congruency_elecs = congruency_elecs_set - switchType_elecs_set - congruency_switchType_elecs_set

    # Convert sets back to lists if necessary for plotting functions
    overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruencyEffectAndSwitchCost_elecs)
    overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list = list(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs)
    unique_switchType_elecs_list = list(unique_switchType_elecs)
    unique_congruency_elecs_list = list(unique_congruency_elecs)
    print(f'there are {len(unique_congruency_elecs_list)} unique congruency elecs')
    print(f'there are {len(unique_switchType_elecs_list)} unique switch type elecs')
    print(f'there are {len(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list)} mixed type elecs')

    transparency = 0.6
    # Plotting
    # Plot unique electrodes for switch type in one color
    fig = plot_on_average(subjects, picks=unique_congruency_elecs_list, rm_wm=False, hemi='both', color=(1,0,0), size=0.35, label_every=None, transparency=transparency)

    # Plot unique electrodes for switchType:switchProportion in another color
    fig = plot_on_average(subjects, picks=unique_switchType_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 1), size=0.35, fig=fig, transparency=transparency)

    # Plot overlapping electrodes in black
    fig = plot_on_average(subjects, picks=overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=transparency)

there are 15 unique congruency elecs
there are 18 unique switch type elecs
there are 15 mixed type elecs


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:822: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot con vs switch using names. Old as of 5/21.

In [78]:
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103'] # don't hard code this, delete this line after cns 4/9
congruency_elecs_set = set(congruency_elecs)
switchType_elecs_set = set(switchType_elecs)
congruency_switchType_elecs_set = set(congruency_switchType_elecs)

# Find overlapping electrodes (present in both lists). Then also find union with the interaction effect.
overlap_congruencyEffectAndSwitchCost_elecs = switchType_elecs_set.intersection(congruency_elecs_set)
overlap_and_interaction_congruencyEffectAndSwitchCost_elecs = (overlap_congruencyEffectAndSwitchCost_elecs | congruency_switchType_elecs_set)

# Find unique electrodes in switchType_elecs
unique_switchType_elecs = switchType_elecs_set - congruency_elecs_set - congruency_switchType_elecs_set

# Find unique electrodes in congruency_elecs
unique_congruency_elecs = congruency_elecs_set - switchType_elecs_set - congruency_switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruencyEffectAndSwitchCost_elecs)
overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list = list(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs)

unique_switchType_elecs_list = list(unique_switchType_elecs)
unique_congruency_elecs_list = list(unique_congruency_elecs)

# #convert indices back to channel names because indices are broken for plotting right now 4/2. Just remove the indices step entirely later.
# unique_switchType_elecs_names = [ch_names[i] for i in unique_switchType_elecs]
# unique_congruency_elecs_names = [ch_names[i] for i in unique_congruency_elecs]
# overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names = [ch_names[i] for i in overlap_and_interaction_congruencyEffectAndSwitchCost_elecs]

# def format_subject_names(subject_names):
#     formatted_names = []
#     for name in subject_names:
#         # Extract the numeric part of the subject name (assuming it always starts with 'D')
#         num_part = int(name[1:])
#         # Format the number with leading zeros to ensure it has four digits and prepend 'D'
#         formatted_name = 'D' + format(num_part, '04')
#         formatted_names.append(formatted_name)
#     return formatted_names

# # For your specific cases of electrode names where the subject name is part of the electrode name
# def format_electrode_names(electrode_names):
#     formatted_electrode_names = []
#     for name in electrode_names:
#         # Split the electrode name at the first '-' to separate subject name from the rest
#         parts = name.split('-', 1)
#         subject_name = parts[0]
#         rest_of_name = parts[1] if len(parts) > 1 else ''
#         # Format the subject name
#         formatted_subject_name = format_subject_names([subject_name])[0]
#         # Combine the formatted subject name with the rest of the electrode name
#         formatted_electrode_name = formatted_subject_name + '-' + rest_of_name
#         formatted_electrode_names.append(formatted_electrode_name)
#     return formatted_electrode_names

# # Example usage:
# unique_switchType_elecs_names_formatted = format_electrode_names(unique_switchType_elecs_names)
# unique_congruency_elecs_names_formatted = format_electrode_names(unique_congruency_elecs_names)
# overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names_formatted = format_electrode_names(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names)

# print(unique_congruency_elecs_names_formatted)

# # Plotting
# # Plot unique electrodes for congruency effect
# fig = plot_on_average(subjects, picks=unique_congruency_elecs_names_formatted, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.45, transparency=0.4)

# # Plot unique electrodes for switchType in another color
# fig = plot_on_average(subjects, picks=unique_switchType_elecs_names_formatted, rm_wm=False, hemi='both', color=(0, 0, 1), size=0.45, fig=fig)

# # Plot overlapping electrodes and interaction electrodes in black
# fig = plot_on_average(subjects, picks=overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_names_formatted, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.45, fig=fig)

NameError: name 'ch_names' is not defined

In [ ]:
unique_switchType_elecs_list

In [ ]:
len(unique_congruency_elecs_list)

In [ ]:
len(overlap_and_interaction_congruencyEffectAndSwitchCost_elecs_list)

plot congruency_switchType interaction vs intersection of congruency and switch type  
i am aware these variable names are awful and inconsistent. Change them later.. 3/27

In [ ]:
congruency_switchType_elecs_set = set(congruency_switchType_elecs)

# Find overlapping electrodes (present in both lists)
overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs = congruency_switchType_elecs_set.intersection(overlap_congruencyEffectAndSwitchCost_elecs)

# Find unique electrodes in conSwitchInteraction
unique_congruency_switchType_elecs = congruency_switchType_elecs_set - overlap_congruencyEffectAndSwitchCost_elecs

# Find unique electrodes in overlap_congruencyEffectAndSwitchCost_elecs
unique_overlap_congruencyEffectAndSwitchCost_elecs = overlap_congruencyEffectAndSwitchCost_elecs - congruency_switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs)
unique_congruency_switchType_elecs_list = list(unique_congruency_switchType_elecs)
unique_overlap_congruencyEffectAndSwitchCost_elecs_list = list(unique_overlap_congruencyEffectAndSwitchCost_elecs)

# Plotting
# Plot unique electrodes for con switch interaction
fig = plot_on_average(subjects, picks=unique_congruency_switchType_elecs_list, rm_wm=True, hemi='both', color=(0.5, 0.3, 0.1), size=0.35, transparency=0.2)

# Plot unique electrodes for con switch overlap
fig = plot_on_average(subjects, picks=unique_overlap_congruencyEffectAndSwitchCost_elecs_list, rm_wm=True, hemi='both', color=(1, 1, 1), size=0.35, fig=fig, transparency=0.2)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_congruency_switchType_and_overlap_congruencyEffectAndSwitchCost_elecs_list, rm_wm=True, hemi='both', color=(0, 0, 0), size=0.35, fig=fig, transparency=0.2)

plot congruency switch interaction and additive effects VS congruency and switch type unique main effects

In [79]:
conSwitchInteractionOrAdditiveEffects = (set(congruency_switchType_interaction_elecs) | set(overlap_congruencyEffectAndSwitchCost_elecs))
conSwitchMainEffects = (set(congruency_elecs) | set(switchType_elecs))

# Find overlapping electrodes (present in both lists)
overlapMainAndInteractionEffects = conSwitchInteractionOrAdditiveEffects.intersection(conSwitchMainEffects)

# Find unique electrodes in conSwitchInteractionOrAdditiveEffects
unique_conSwitchInteractionOrAdditiveEffects_elecs = conSwitchInteractionOrAdditiveEffects - conSwitchMainEffects

# Find unique electrodes in conSwitchMainEffects
unique_conSwitchMainEffects_elecs = conSwitchMainEffects - conSwitchInteractionOrAdditiveEffects

# Convert sets back to lists if necessary for plotting functions
overlapMainAndInteractionEffects_list = list(overlapMainAndInteractionEffects)
unique_conSwitchInteractionOrAdditiveEffects_elecs_list = list(unique_conSwitchInteractionOrAdditiveEffects_elecs)
unique_conSwitchMainEffects_elecs_list = list(unique_conSwitchMainEffects_elecs)

# Plotting
# Plot unique electrodes for con switch interaction
fig = plot_on_average(subjects, picks=unique_conSwitchInteractionOrAdditiveEffects_elecs_list, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.45, transparency=0.2)

# Plot unique electrodes for con switch overlap
fig = plot_on_average(subjects, picks=unique_conSwitchMainEffects_elecs_list, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.45, fig=fig, transparency=0.2)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlapMainAndInteractionEffects_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.45, fig=fig, transparency=0.2)

NameError: name 'congruency_switchType_interaction_elecs' is not defined

In [ ]:
len(overlapMainAndInteractionEffects_list)

### plot all stim electrodes for an roi
get the subject: electrode mappings from roi_analysis first

In [ ]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

In [ ]:
import json

# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

In [ ]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

this is with zeroes removed from subject names, i think plot on average needs those instead of the normal subject names  
also, remove the subjects with empty lists since those might be breaking the plotting too

In [ ]:
def remove_leading_zeroes_and_empty_subjects(subjects_dict):
    """
    Removes leading zeroes from subject names, adds a '_without_zeroes' suffix, 
    and filters out subjects with empty lists.

    Args:
    subjects_dict (dict): A dictionary with subjects as keys and lists as values.

    Returns:
    dict: A new dictionary with modified subject names and non-empty lists.
    """
    new_dict = {}
    for sub, value in subjects_dict.items():
        if value:  # Check if the list is not empty
            sub_without_zeroes = "D" + sub[1:].lstrip('0')
            new_dict[sub_without_zeroes] = value
    return new_dict

# Example usage:
dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(dlpfc_electrodes_per_subject)
sig_dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_dlpfc_electrodes_per_subject)

acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(acc_electrodes_per_subject)
sig_acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_acc_electrodes_per_subject)

parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(parietal_electrodes_per_subject)
sig_parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_parietal_electrodes_per_subject)

### plot sig roi electrodes on individual brains

#### dlpfc

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')
# brain_fig = plot_subj(subj_without_zeroes, picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig dlpfc electrodes

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    if subj == 'D57':
        continue #skip this one cuz its being weird
    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### acc

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig acc electrodes

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### parietal

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig parietal electrodes

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

### do dlpfc, acc, parietal coverage for each person

get all electrodes across dlpfc, acc, and parietal and stick them into one dictionary so we can plot coverage in control regions for each subject

In [ ]:
allSigControlElectrodes = {key: sig_parietal_electrodes_without_zeroes.get(key, []) + 
                      sig_dlpfc_electrodes_without_zeroes.get(key, []) + 
                      sig_acc_electrodes_without_zeroes.get(key, [])
                 for key in set(sig_parietal_electrodes_without_zeroes) | 
                             set(sig_dlpfc_electrodes_without_zeroes) | 
                             set(sig_acc_electrodes_without_zeroes)}

allControlElectrodes = {key: parietal_electrodes_without_zeroes.get(key, []) + 
                      dlpfc_electrodes_without_zeroes.get(key, []) + 
                      acc_electrodes_without_zeroes.get(key, [])
                 for key in set(parietal_electrodes_without_zeroes) | 
                             set(dlpfc_electrodes_without_zeroes) | 
                             set(acc_electrodes_without_zeroes)}

subj = 'D63'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=allSigControlElectrodes[subj_without_zeroes], nonsig_picks=allControlElectrodes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_allControlElectrodes', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_allControlElectrodes.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

### try to plot all sig electrodes in an roi on an avg brain  
this is plotting like all electrodes wtf why

In [ ]:
plot_on_average(sig_parietal_electrodes_without_zeroes, hemi='both', color=(1,1,0))

### plot each subject one by one on the fsaverage brain

In [ ]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]
allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

subj = 'D0071'
    
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)

# Combine the significant electrodes for this subject
combined_chans = resp_sig_chans + stim_sig_chans

# Add them to the dictionary, tagged with the subject without zeroes

subj_without_zeroes = "D" + subj[1:].lstrip('0')
allSigElectrodes[subj_without_zeroes] = combined_chans

# Get all electrodes for this subject and add them to the allElectrodes dict
info = subject_to_info(subj_without_zeroes)
allSubjElectrodes = mne.pick_channels(info.ch_names, [])
allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

just plot an empty brain for my MAP 3/21

In [28]:
import os
import mne
from mne.viz import Brain

subj = 'D57'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
# subjects_dir = get_sub_dir()  # Use your default subjects directory function
sub_dir = get_sub_dir()

# Create a brain directly using the Brain class instead of plot_subj
brain_fig = Brain(
    subject=subj,
    hemi='rh',
    background='white',
    offset=True,
    alpha=0.9,  # Very transparent (0.6 is quite transparent)
    subjects_dir=sub_dir
)

# Set to top view (dorsal view)
brain_fig.show_view('lateral')

# Specify the path for output
output_path = os.path.join(base_path, f"{subj}_transparent_rh_lateral_brain_no_electrodes.png")

# Save the image
brain_fig.save_image(output_path)
print(f"Brain visualization saved to {output_path}")

Brain visualization saved to C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D57_transparent_rh_lateral_brain_no_electrodes.png


In [30]:
import os
import mne
from mne.viz import Brain

subj = 'D57'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
sub_dir = get_sub_dir()

# Define a custom cortex style with very subtle coloring
# Format is (color for sulci, color for gyri)
white_cortex = ('white', '#F8F8F8')  # Nearly white for both, with subtle difference

# Create a brain with mostly white appearance
brain_fig = Brain(
    subject=subj,
    hemi='rh',
    background='white',
    cortex=white_cortex,  # Custom coloring
    alpha=1.0,  # Fully opaque since we're using white anyway
    subjects_dir=sub_dir
)

# Set to lateral view
brain_fig.show_view('medial')

# Turn on silhouette to emphasize the outline
brain_fig.silhouette = True

# Specify the path for output
output_path = os.path.join(base_path, f"{subj}_white_outline_rh_medial.png")

# Save the image
brain_fig.save_image(output_path)
print(f"White outline brain visualization saved to {output_path}")

White outline brain visualization saved to C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D57_white_outline_rh_medial.png
